<a href="https://colab.research.google.com/github/Ra-ha/Bangla_News_Categorization/blob/main/BANGLA_NEWS_CATEGORIZATION_USING%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/root/gdrive', force_remount=True)

Mounted at /root/gdrive


In [ ]:
%cd /root/gdrive/MyDrive/

/root/gdrive/MyDrive


Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import Counter

Load data

In [ ]:

with open('data.json', encoding='utf-8') as fh:
    orig_data = json.load(fh)
print(orig_data[0])

Trim data for test run

In [ ]:
data = orig_data[:5000]

NameError: ignored

Visualize data

In [ ]:
print("Total Rows:", len(data))
print("Total Cols:", len(data[0]))

Total Rows: 23000
Total Cols: 10


Show all category

In [ ]:
print(type(data))
print("All columns:")
for col in data[0]:
  print(col)

<class 'list'>
All columns:
author
category
category_bn
published_date
modification_date
tag
comment_count
title
url
content


Retrieve all categories

In [ ]:
all_categories = set([sample['category'] for sample in data])
print(all_categories)
print(len(all_categories))

{'education', 'opinion', 'bangladesh', 'sports', 'international', 'AskEditor', 'life-style', 'roshalo', 'kishoralo', 'events', 'entertainment', 'facebook', 'economy', 'we-are', 'technology', 'durporobash', 'onnoalo', '-1', 'pachmisheli'}
19


test code

Sorting based on category count

In [ ]:
category_count_list = []
count = 0
for cat in all_categories:
  count = 0
  for row in data:
    if row['category'] == cat:
      count += 1
  category_count_list.append({'category': cat, 'count': count})
print(category_count_list)

from operator import itemgetter
sorted_cat_count_list = []
# sorted_cat_count_list = sorted(category_count_list, key=lambda x: x.count, reverse=True)
category_count_list.sort(key = itemgetter('count'), reverse=True)
print(category_count_list)

[{'category': 'education', 'count': 745}, {'category': 'opinion', 'count': 752}, {'category': 'bangladesh', 'count': 12226}, {'category': 'sports', 'count': 2702}, {'category': 'international', 'count': 1763}, {'category': 'AskEditor', 'count': 1}, {'category': 'life-style', 'count': 443}, {'category': 'roshalo', 'count': 142}, {'category': 'kishoralo', 'count': 1}, {'category': 'events', 'count': 2}, {'category': 'entertainment', 'count': 1616}, {'category': 'facebook', 'count': 9}, {'category': 'economy', 'count': 827}, {'category': 'we-are', 'count': 357}, {'category': 'technology', 'count': 1020}, {'category': 'durporobash', 'count': 1}, {'category': 'onnoalo', 'count': 168}, {'category': '-1', 'count': 2}, {'category': 'pachmisheli', 'count': 223}]
[{'category': 'bangladesh', 'count': 12226}, {'category': 'sports', 'count': 2702}, {'category': 'international', 'count': 1763}, {'category': 'entertainment', 'count': 1616}, {'category': 'technology', 'count': 1020}, {'category': 'eco

Discard low count catagories

In [ ]:
selected_cats = []

for p in category_count_list:
    if p['count'] > 300:
        selected_cats.append(p['category'])
print(selected_cats)
print(len(selected_cats))

['bangladesh', 'sports', 'international', 'entertainment', 'technology', 'economy', 'opinion', 'education', 'life-style', 'we-are']
10


Prepare data X, Y

In [ ]:
x_text = []
y_label = []

for row in data:
    if row['category'] in selected_cats:
        y_label.append(row['category'])
        x_text.append(row['content'])
print(len(x_text),len(y_label))

22451 22451


Split data into Train & test set

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
 x_text, y_label, test_size=0.33, random_state=42)

print(len(X_train),len(X_test))

15042 7409


xlm Roberta

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
!pip install transformers
import transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, TFAutoModel, TFXLMRobertaModel

bert_tokenizer = AutoTokenizer.from_pretrained('jplu/tf-xlm-roberta-base')
# bert_model = TFAutoModel.from_pretrained("jplu/tf-xlm-roberta-large")
bert_model = TFXLMRobertaModel.from_pretrained('jplu/tf-xlm-roberta-base')

Some layers from the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Tokenize x

In [ ]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = bert_tokenizer(
    text=X_train,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = bert_tokenizer(
    text=X_test,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

One hot encoding of Y

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def make_ohe(y_label):
  # Label Encode
  encoder = LabelEncoder()
  class_labels = encoder.fit_transform(y_label)
  print(class_labels,len(class_labels),class_labels.shape)
  print(set(class_labels))

  #One hot Encode
  encoder = OneHotEncoder(sparse=False)
  class_labels = class_labels.reshape((class_labels.shape[0], 1))
  y_ohe = encoder.fit_transform(class_labels)
  print(y_ohe,y_ohe.shape)
  return y_ohe

In [ ]:
y_train = make_ohe(y_train)
print(type(y_train))

[0 3 0 ... 7 0 0] 15042 (15042,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] (15042, 10)
<class 'numpy.ndarray'>


In [ ]:
y_test = make_ohe(y_test)
print(type(y_test))

[4 0 3 ... 0 0 3] 7409 (7409,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (7409, 10)
<class 'numpy.ndarray'>


In [ ]:
max_len = 512
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert_model(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
# out = Dense(128, activation='relu')(out)
# out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(10,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tfxlm_roberta_model_2 (TFXLMRo  TFBaseModelOutputWi  278043648  ['input_ids[0][0]',              
 bertaModel)                    thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
# loss =CategoricalCrossentropy(from_logits = True)
# metrics = CategoricalAccuracy('balanced_accuracy'),
loss='categorical_crossentropy'
metrics=['accuracy']
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metrics)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('xlmRoberta_22k.h5', monitor='loss', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)

In [ ]:
train_history = model.fit(x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']},validation_split=0.2,callbacks=[checkpoint, earlystopping],y= y_train, epochs=5, batch_size =6)

NameError: ignored

In [ ]:
model.save("xlmRoberta_22k.h5")

In [ ]:
from sklearn.metrics import classification_report

Train accuracy

In [ ]:
predicted_raw_train = model.predict({'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']})
predicted_raw_train[0]

In [ ]:
y_predicted_train = np.argmax(predicted_raw_train, axis = 1)
y_true_train = np.argmax(y_train, axis = 1)
print(classification_report(y_true_train, y_predicted_train))

Test accuracy

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = np.argmax(y_test, axis = 1)
print(classification_report(y_true, y_predicted))

Done

In [ ]:
history = model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()